In [1]:
# combine all the file into one - Done

# write the function to take lookback window from 2 to 14 and calculate the corresponding pnl  - Done

# check each sector's pnl (later)

# add interaction as the threshold - Done

# remove all the extra strings  - Done

In [1]:
cd January

/Users/herculesgao/Desktop/Information_Tracer/January


In [2]:
import glob
import pandas as pd
import numpy as np


df_stock = pd.DataFrame()
for file_name in glob.glob('SPY500_stock*2024*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    df_stock = pd.concat([df_stock,x],axis=0)
df_stock = df_stock.drop(columns = 'Unnamed: 0')
df_stock.sort_values(['Symbol', 'Date'], inplace=True)

df_temp = df_stock.groupby(['Symbol']).agg(numStock = ('Symbol', 'count')).reset_index()
days = df_stock['Date'].nunique()

excludedSymbol = list(df_temp[df_temp['numStock']< days]['Symbol'])
print("we need to remove the following stocks", excludedSymbol)
df_stock = df_stock[~df_stock['Symbol'].isin(excludedSymbol)].reset_index(drop= True)

we need to remove the following stocks ['AEE', 'BG', 'IEX', 'MHK', 'MLM', 'TDY', 'VLTO', 'VMC', 'WTW']


In [3]:
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import requests
import bs4 as bs

analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score


# change tweets 
def tweets_processing(df_stock):
    df_stock['Sentiment_Vader_corrected'] = ''
    for j in range(len(df_stock['Posts_t'])):
        tweets = df_stock.iloc[j]['Posts_t']

        if len(tweets) >2:
            tweets = tweets.split("},")
    #         print(tweets)
            for i in range(len(tweets)):
                test_tw = re.split(" https://t",re.split("'d': |'i':",tweets[i])[1][1:-1])[0] # remove html and extra information

                test_tw = ("").join(test_tw.split("\\n"))  #remove newline 
                test_tw = ("").join(test_tw.split("amp;"))  # remove ampersand
                new_score = sentiment_analyzer_scores(test_tw)['compound']
            df_stock.loc[j,'Sentiment_Vader_corrected'] = new_score
        else:
            df_stock.loc[j,'Sentiment_Vader_corrected'] = 0.0
    return df_stock

def look_back_sentiment(df_stock, org = True, lookback_window=1):
    n = df_stock['Date'].nunique()
    Sentiment_Vader_2 = []
    for i in range(0,len(df_stock['Posts_t']),n):
        for j in range(n):
            if (j - lookback_window - 1 ) >=0:
                past_days = df_stock.iloc[i+j-lookback_window:i+j]
                if org:
                    sentiment_now =  past_days[past_days['Posts_t'].str.len()>2]['Sentiment_Vader'].mean()
                else:
                    sentiment_now =  past_days[past_days['Posts_t'].str.len()>2]['Sentiment_Vader_corrected'].mean()
                if np.isnan(sentiment_now):  #find all the empty twitter list
                    Sentiment_Vader_2.append(0.0)
                else:         
                    Sentiment_Vader_2.append(sentiment_now)
            else:  # outside the lookback window days 
                #base case 
                Sentiment_Vader_2.append(0.0)
    df_stock['Sentiment_Vader_2'] = Sentiment_Vader_2
    return df_stock

df_stock = tweets_processing(df_stock)


In [5]:


def VADER_Sentiment_report(df_stock, lookback_window = 1, interact_threshold = 0, org = True):
    pnl = 0 
    stock_pnl = []
    stock_names = []
    len_tweets = []
    interactions = []
    sumA = 0 
    sumTP = 0 
    sumTN = 0
    sumFP = 0 
    sumFN = 0 
    sumWrong = 0 
    sumPlus = 0 
    df_stock = look_back_sentiment(df_stock, org, lookback_window)
    n = df_stock['Date'].nunique()
    for i in range(0,len(df_stock['Symbol']),n): 
        symbol = df_stock['Symbol'][i]
        temp_hold = 0
        long = False
        short = False
        total_pnl = 0 
        total_t = 0
        num = 0 
        num_s = 0 
        interaction = 0 
        for j in range(n):
            change = df_stock['Changes'][i+j]
            interaction += df_stock['interaction'][i+j]
            tw = df_stock['Posts_t'][i+j][1:-1].split("},")
#             print(tw, len(tw))
            num_tw = len(df_stock['Posts_t'][i+j][1:-1].split("},"))
            if num_tw >= 1 and len(df_stock['Posts_t'][i+j][1:-1].split("},")[0])>0:
                total_t += num_tw
            else:
                total_t += 0
            if df_stock['Sentiment_Vader_2'][i+j] >= 0.05 and df_stock['interaction'][i+j] >= interact_threshold:
                # no position
                if not long and not short:  # the first buy
                    temp_hold = df_stock['Adj Close'][i+j]
                    num+=1
                    long = True
                # have a short position and close the short position
                elif short: 
                    total_pnl += (temp_hold + df_stock['Adj Close'][i+j])
                    short = False
                    num_s = 0 
                elif long and not short: 
                    num+=1
                    temp_hold += df_stock['Adj Close'][i+j]

            elif df_stock['Sentiment_Vader_2'][i+j] <= -0.05 and df_stock['interaction'][i+j] >= interact_threshold:
                # have a long position and close the long position 
                if long and not short: 
#                         print("sell signal and exit")
                    total_pnl += (df_stock['Adj Close'][i+j] -  (temp_hold/num))*1
                    long = False
                    num = 0 
                # do not have long or short position
                elif not long and not short: 
                    temp_hold = - df_stock['Adj Close'][i+j]
                    short = True
                    num_s = 1
#                 # do not have a long position and already short a stock
                elif not long and short: 
#                         print("short again")
                    temp_hold = (- df_stock['Adj Close'][i+j]+ temp_hold)/(num_s+1)
                    num_s +=1

            if long and not short and j == n-1: 
                total_pnl += (df_stock['Adj Close'][i+j] - (temp_hold/num))*1
            elif short and not long and j == n-1: 
                total_pnl += ((df_stock['Adj Close'][i+j] + temp_hold)/(num_s+1))*1
            if change > 0: sumPlus += 1
            if (df_stock['Sentiment_Vader_2'][i+j] >= 0.05 and change > 0) or (df_stock['Sentiment_Vader_2'][i+j] <= - 0.05  and float(change) < 0): 
                sumA += 1
                if df_stock['Sentiment_Vader_2'][i+j] >= 0.05 and change > 0:
                    sumTP +=1
                else:
                    sumTN +=1
            if (df_stock['Sentiment_Vader_2'][i+j] >=  0.05  and change < 0) or (df_stock['Sentiment_Vader_2'][i+j] <= - 0.05  and float(change) > 0): 
                sumWrong += 1
                if df_stock['Sentiment_Vader_2'][i+j] >= 0.05 and change < 0:
                    sumFP +=1
                else:
                    sumFN +=1
        stock_names.append(df_stock['Symbol'][i])
        len_tweets.append(total_t)
        interactions.append(interaction/n)
        pnl += total_pnl
        stock_pnl.append(total_pnl)
    stock = sorted(set(df_stock['Symbol']))
    df_pnl = pd.DataFrame({'Symbol': stock_names, 'PNL_VADER':stock_pnl, 'Len_Ts':len_tweets,'Interactions':interactions} )
#         print(sumPlus,sumA, sumWrong)
    l_hProfits = []
    for i in range(0,len(df_stock['Symbol']),n): 
        l_hProfit = (df_stock['Adj Close'][i+n-1] - df_stock['Adj Close'][i])
        l_hProfits.append(l_hProfit)
    df_pnl['PNL_LongnHold'] = l_hProfits
    
    # add sectors 
    
    
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    table.findAll('tr')[1:][0].findAll('td')[1].text
    tickers = []
    sectors = []
    dc = {}

    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        sector = row.findAll('td')[2].text
        dc[ticker.replace('\n', '').replace('.', '-')] = sector

    extra_list = []

    sectors = []
    for i in df_pnl['Symbol']:
        try:
    #         print(i,dc[i])
            if i == 'CDAY':
              sectors.append('Information Technology') 
            else:
                sectors.append(dc[i])
        except: 
            
            sec = """
            Sector Reference table: 

             'Health Care',
             'Industrials',
             'Information Technology',
             'Consumer Discretionary',
             'Financials',
             'Consumer Staples',
             'Utilities',
             'Materials',
             'Real Estate',
             'Energy',
             'Communication Services'

             """
            print("we could not find the sector for the following!",i)
            print(sec)
            add_sector = input("Please give a sector for the ticker below!")
            
            extra_list.append((i,add_sector))
            sectors.append(add_sector)
    df_pnl['Sector'] = sectors
    total_vader_num = len(df_stock[df_stock['Sentiment_Vader_corrected']!=0])
    accuracy = sumA/total_vader_num
    
    f1 = sumTP/(sumTP + 0.5*(sumFP + sumTN))

    return pnl, df_pnl,accuracy,f1


In [6]:
lookback_days = []
accuracys = []
f1s = []
pnls = []
orgins = []
interactions_thres = []
total_pnldfs = []
for i in range(2,15):
    print("Different days (between 2-14)", i)
    lookback_days.append(i)
    print("Orginal", VADER_Sentiment_report(df_stock, i, 0, True)[0])
    pnl, df_pnl,accuracy,f1 = VADER_Sentiment_report(df_stock,i, 0, True)
    accuracys.append(accuracy)
    f1s.append(f1)
    pnls.append(pnl)
    interactions_thres.append(0)
    orgins.append(True)
    total_pnldfs.append(df_pnl)
    
    print("Text-preprocessed", VADER_Sentiment_report(df_stock, i, 0, False)[0])
    lookback_days.append(i)
    pnl, df_pnl,accuracy,f1 = VADER_Sentiment_report(df_stock,i, 0, False)
    accuracys.append(accuracy)
    f1s.append(f1)
    pnls.append(pnl)
    interactions_thres.append(0)
    orgins.append(False)
    total_pnldfs.append(df_pnl)
    
df_sum = pd.DataFrame({'lookback_days':lookback_days,
                        'accuracy':accuracys,'f1':f1s,
                        'pnl':pnls,
                        'orgin':orgins,
                        'interactions_thres':interactions_thres})
# VADER_Sentiment_report(df_stock, 4, 0)[0]

# VADER_Sentiment_report(df_stock, 4, 0)[0]

Different days (between 2-14) 2
Orginal 4230.853093657249
Text-preprocessed 4302.6659721025835
Different days (between 2-14) 3
Orginal 5374.246510081203
Text-preprocessed 6425.9504977816605
Different days (between 2-14) 4
Orginal 5394.955937221596
Text-preprocessed 5784.288831980907
Different days (between 2-14) 5
Orginal 4352.084328376773
Text-preprocessed 4483.920020657266
Different days (between 2-14) 6
Orginal 3260.9251871133997
Text-preprocessed 3241.778959848425
Different days (between 2-14) 7
Orginal 3094.3098278053476
Text-preprocessed 2651.7160412515013
Different days (between 2-14) 8
Orginal 2396.382282763568
Text-preprocessed 2158.8261920891273
Different days (between 2-14) 9
Orginal 1929.191441588082
Text-preprocessed 2035.253586706425
Different days (between 2-14) 10
Orginal 1151.3083473602935
Text-preprocessed 1658.240196939121
Different days (between 2-14) 11
Orginal 639.0470395088195
Text-preprocessed 1195.286784063445
Different days (between 2-14) 12
Orginal 502.403024

In [26]:
df_sum


,lookback_days,accuracy,f1,pnl,orgin,interactions_thres
0,2,0.641979,0.634593,4230.853094,True,0
1,2,0.515111,0.616766,4302.665972,False,0
2,3,0.655264,0.645911,5374.246510,True,0
3,3,0.548987,0.630359,6425.950498,False,0
4,4,0.605115,0.620744,5394.955937,True,0
5,4,0.527067,0.604225,5784.288832,False,0
6,5,0.594155,0.644581,4352.084328,True,0
7,5,0.520757,0.635442,4483.920021,False,0
8,6,0.537363,0.647138,3260.925187,True,0
9,6,0.478246,0.636198,3241.778960,False,0


In [8]:
print("Long and Hold pnl", sum(df_pnl['PNL_LongnHold']))

Long and Hold pnl 1533.0846652984621


In [9]:
# 3 the most profitable one
pnl, df_pnl,accuracy,f1 = VADER_Sentiment_report(df_stock, 3, 0, False)

In [10]:
df_pnl

,Symbol,PNL_VADER,Len_Ts,Interactions,PNL_LongnHold,Sector
0,A,0.444997,9,0.176471,-7.809998,Health Care
1,AAL,5.916667,166,47.470588,1.690001,Industrials
2,AAPL,1.597140,3686,5035.058824,6.779999,Information Technology
3,ABBV,2.013574,153,53.882353,4.539993,Health Care
4,ABNB,4.590004,93,14.411765,15.139999,Consumer Discretionary
...,...,...,...,...,...,...
489,YUM,17.024197,152,19.000000,0.019989,Consumer Discretionary
490,ZBH,-0.372728,18,2.882353,0.349998,Health Care
491,ZBRA,84.266995,12,1.294118,-15.810013,Information Technology
492,ZION,1.595556,25,1.000000,-0.119999,Financials


In [11]:
sum(df_pnl['PNL_LongnHold']),sum(df_pnl['PNL_VADER'])

(1533.0846652984621, 6425.9504977816605)

In [13]:
def get_df_summary(df,df_stock):
    l_hProfits = []
    pctVaderPNL = []
    pctLHPNL = []
    total_cost = 0

    nuni = df_stock['Date'].nunique()
    for i in range(0,len(df_stock['Symbol']),nuni): 
    #     print(df_stock['Symbol'][i], df_stock['Symbol'][i+5])
        l_hProfit = df_stock['Adj Close'][i+nuni-1] - df_stock['Adj Close'][i]
        l_hProfits.append(l_hProfit)
      

    df['PNL_LongnHold'] = l_hProfits
    
    for j in range(0,len(df_stock['Symbol']),nuni): 
        stock_vader = df[df['Symbol']==df_stock['Symbol'][j]]['PNL_VADER']
        stock_lh = df[df['Symbol']==df_stock['Symbol'][j]]['PNL_LongnHold']
        pnl = stock_vader.loc[stock_vader.index[0]]
        pnl_lh = stock_lh.loc[stock_lh.index[0]]
        pctVaderPNL.append((pnl/df_stock['Adj Close'][j])*100)
        pctLHPNL.append((pnl_lh/df_stock['Adj Close'][j])*100)
        total_cost += df_stock['Adj Close'][i]
    print("PNL of VADER", sum(df['PNL_VADER']))
    print("PNL of L/H", sum(df['PNL_LongnHold']))

    df['pctVaderPNL'] = pctVaderPNL
    df['pctLHPNL'] = pctLHPNL
    
    print("PNL Percentage of VADER", np.mean(df['pctVaderPNL']))
    print("PNL Percentage of L/H", np.mean(df['pctLHPNL']))
    
    df_summary = df.groupby('Sector', group_keys=True, as_index = False).agg(
        avgPctVADER=('pctVaderPNL', np.mean),
        avgPctL_H=('pctLHPNL', np.mean),
        sumVADER=('PNL_VADER', np.sum),
        sumL_H=('PNL_LongnHold', np.sum),
        sumTwts =('Len_Ts', np.sum),
        CountStocks=('Symbol', 'nunique'))
    return df, df_summary,total_cost, (np.mean(df['pctVaderPNL'])>np.mean(df['pctLHPNL']))

In [14]:
df, df_summary,total_cost,res = get_df_summary(df_pnl,df_stock)

PNL of VADER 6425.9504977816605
PNL of L/H 1533.0846652984621
PNL Percentage of VADER 6.125287734994037
PNL Percentage of L/H -0.05509391575462979


In [15]:
total_cost

97120.4030151367

In [16]:
df_summary

,Sector,avgPctVADER,avgPctL_H,sumVADER,sumL_H,sumTwts,CountStocks
0,Communication Services,8.326234,3.944592,184.399202,191.359985,9840,22
1,Consumer Discretionary,9.473581,-1.026558,1305.754105,410.584185,20292,52
2,Consumer Staples,3.908864,-2.273021,179.667777,-22.700048,2494,37
3,Energy,2.514713,-1.692500,42.762502,-7.069887,2402,23
4,Financials,6.884075,1.856100,666.453157,239.146339,8165,71
5,Health Care,4.316316,-0.800106,918.842927,-93.411609,5837,64
6,Industrials,7.245268,1.014411,1104.638247,193.169567,10750,75
7,Information Technology,11.127089,4.474234,1791.360035,977.149921,14671,64
8,Materials,3.897118,-4.093485,227.224102,-97.778750,3336,26
9,Real Estate,1.563676,-4.300390,23.698046,-169.645008,1361,31


In [21]:
# pctage of pnl 
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# for i in range(len(summary)):
week = 3

labels = list(df_summary["Sector"])

colors = ['green','red','wheat','powderblue','salmon','brown','pink','darkblue','sienna','lightgreen','lightskyblue']
fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5],
                    subplot_titles=("VADER PnL by Sectors with Sentiment Threshold = ±0.1","L/H PnL by Sectors"),
                    vertical_spacing=0.1,
                  horizontal_spacing=0.1,
                  shared_yaxes='all')
fig.add_trace(go.Bar(x = df_summary["Sector"], y=df_summary["avgPctVADER"], marker_color=colors),
              row=1, col=1)

fig.add_trace(go.Bar(x = df_summary["Sector"], y=df_summary["avgPctL_H"],marker_color=colors),
              row=1, col=2)
fig.update_traces(texttemplate='%{y:.2f}')
fig.update_layout(title="Lookback Days-"+str(week)+': PNL percentage comparison',
                   xaxis_title='Sector Name',
                   yaxis_title='PnL ($)')
fig.show()

In [22]:
# pnl 

week = 3

labels = list(df_summary["Sector"])
colors = ['green','red','wheat','powderblue','salmon','brown','pink','darkblue','sienna','lightgreen','lightskyblue']
fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5],
                    subplot_titles=("VADER PnL by Sectors with Sentiment Threshold = ±0.1","L/H PnL by Sectors"),
                    vertical_spacing=0.1,
                  horizontal_spacing=0.1,
                  shared_yaxes='all')
fig.add_trace(go.Bar(x = df_summary["Sector"], y=df_summary["sumVADER"], marker_color=colors),
              row=1, col=1)

fig.add_trace(go.Bar(x = df_summary["Sector"], y=df_summary["sumL_H"],marker_color=colors),
              row=1, col=2)
fig.update_traces(texttemplate='%{y:.2f}')
fig.update_layout(title="Lookback Days-"+str(week)+': PNL comparison',
                   xaxis_title='Sector Name',
                   yaxis_title='PnL ($)')
fig.show()

In [24]:
fig = px.imshow(df_summary[['avgPctVADER','avgPctL_H','sumVADER','sumL_H','sumTwts','CountStocks']].corr(), text_auto=True, aspect="auto")
fig.update_layout(title="Lookback days-"+str(week)+': All tickers PNL each factor in-between correlation', xaxis_nticks=36)
fig.show()

In [84]:
text = "$A Agilent Technologies current stock price $130\\n\\nLow of $130.01 \\nP/E  (price-to-earnings ratio) 31.09 \\nAnnuel EPS grow past 5 years 33%\\nEPS Q/Q (quarterly earnings growth) 31%\\n\\nClosed yesterday at $133 - needs to pull back to $113 to close that gap. LT Puts? https://t.co/QJZBNU7kbW'  "



sentiment_analyzer_scores(text)

{'neg': 0.0, 'neu': 0.94, 'pos': 0.06, 'compound': 0.3818}

In [86]:
text1 = "$A Agilent Technologies current stock price $130 Low of $130.01 P/E (price-to-earnings ratio) 31.09 Annuel EPS grow past 5 years 33%\\nEPS Q/Q (quarterly earnings growth) 31% Closed yesterday at $133 - needs to pull back to $113 to close that gap. LT Puts?"

sentiment_analyzer_scores(text1)

{'neg': 0.046, 'neu': 0.897, 'pos': 0.057, 'compound': 0.128}

In [88]:
text1 = "A Agilent Technologies current stock price $130 Low of $130.01 P/E (price-to-earnings ratio) 31.09 Annuel EPS grow past 5 years 33%\\nEPS Q/Q (quarterly earnings growth) 31% Closed yesterday at $133 - needs to pull back to $113 to close that gap. LT Puts?"

sentiment_analyzer_scores(text1)

{'neg': 0.046, 'neu': 0.897, 'pos': 0.057, 'compound': 0.128}